In [1]:
import warnings

warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import plotly.express as px
import sklearn as sk
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df=pd.read_csv("HR_clean.csv")
df.head(10)

,Unnamed: 0,satisfaction_level,last_evaluation_score,project_count,average_monthly_hours,years_of_working,workplace_acciedent_count,employment_status,promotion_last_5years,department,salary_range,Score_variance
0,0,0.38,0.53,2,157,3,0,1,0,sales,low,0.15
1,2,0.11,0.88,7,272,4,0,1,0,sales,medium,0.77
2,3,0.72,0.87,5,223,5,0,1,0,sales,low,0.15
3,4,0.37,0.52,2,159,3,0,1,0,sales,low,0.15
4,5,0.41,0.50,2,153,3,0,1,0,sales,low,0.09
5,6,0.10,0.77,6,247,4,0,1,0,sales,low,0.67
6,7,0.92,0.85,5,259,5,0,1,0,sales,low,-0.07
7,8,0.89,1.00,5,224,5,0,1,0,sales,low,0.11
8,9,0.42,0.53,2,142,3,0,1,0,sales,low,0.11
9,10,0.45,0.54,2,135,3,0,1,0,sales,low,0.09


In [3]:
df.drop(columns=['Unnamed: 0'], inplace=True)
df.columns

Index(['satisfaction_level', 'last_evaluation_score', 'project_count',
       'average_monthly_hours', 'years_of_working',
       'workplace_acciedent_count', 'employment_status',
       'promotion_last_5years', 'department', 'salary_range',
       'Score_variance'],
      dtype='object')

## Method 4 - Use of Ensemble with undersamping 

In [24]:
df= pd.get_dummies(data=df, columns=["department", "salary_range"])
df.columns

Index(['satisfaction_level', 'last_evaluation_score', 'project_count',
       'average_monthly_hours', 'years_of_working',
       'workplace_acciedent_count', 'employment_status',
       'promotion_last_5years', 'Score_variance', 'department_IT',
       'department_RandD', 'department_accounting', 'department_hr',
       'department_management', 'department_marketing',
       'department_product_mng', 'department_sales', 'department_support',
       'department_technical', 'salary_range_high', 'salary_range_low',
       'salary_range_medium'],
      dtype='object')

In [25]:
df.employment_status.value_counts()

0    9285
1    1882
Name: employment_status, dtype: int64

In [26]:
x = df.drop("employment_status", axis=1)
y = df["employment_status"]

In [27]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=15, stratify=y)

In [28]:
y_train.value_counts()

0    7428
1    1505
Name: employment_status, dtype: int64

In [29]:
7428/1505


4.935548172757475

In [30]:
7428/5

1485.6

In [31]:
df1 = x_train.copy()
df1["employment_status"] = y_train

In [32]:
df_class0 = df1[df1.employment_status==0]
df_class1 = df1[df1.employment_status==1]

In [33]:
df_class0.shape, df_class1.shape

((7428, 22), (1505, 22))

In [34]:
df_class0[:1505].shape

(1505, 22)

In [35]:
df_train = pd.concat([df_class0[:1505], df_class1],axis = 0)
df_train.shape

(3010, 22)

In [36]:
def get_train_batch(df_majority, df_minority, start,end):
    df_train = pd.concat([df_majority[start:end], df_minority],axis = 0)
    x_train = df_train.drop("employment_status", axis=1)
    y_train = df_train.employment_status
  
    return x_train,y_train
    

## Perfomance of Logistic regression model (0-1505)

In [40]:
from sklearn.metrics import classification_report

In [41]:
x_train, y_train = get_train_batch(df_class0, df_class1, 0, 1505)
x_train.shape

(3010, 21)

In [45]:
from sklearn.linear_model import LogisticRegression
log1 = LogisticRegression()
log1.fit(x_train,y_train)
log1.score(x_test,y_test)
print(classification_report (y_test, log1.predict(x_test)))

              precision    recall  f1-score   support

           0       0.98      0.83      0.89      1857
           1       0.51      0.90      0.65       377

    accuracy                           0.84      2234
   macro avg       0.74      0.86      0.77      2234
weighted avg       0.90      0.84      0.85      2234



In [58]:
log1.score(x_test,y_test)

0.8384064458370636

## Perfomance of Logistic regression model (1505-3010)

In [46]:
x_train, y_train = get_train_batch(df_class0, df_class1, 1505, 3010)
from sklearn.linear_model import LogisticRegression
log2 = LogisticRegression()
log2.fit(x_train,y_train)
log2.score(x_test,y_test)
print(classification_report (y_test, log2.predict(x_test)))

              precision    recall  f1-score   support

           0       0.98      0.83      0.90      1857
           1       0.52      0.91      0.66       377

    accuracy                           0.84      2234
   macro avg       0.75      0.87      0.78      2234
weighted avg       0.90      0.84      0.86      2234



In [57]:
log2.score(x_test,y_test)

0.842882721575649

## Perfomance of Logistic regression model (3010-4515)

In [49]:
x_train, y_train = get_train_batch(df_class0, df_class1,3010,4515)
from sklearn.linear_model import LogisticRegression
log3 = LogisticRegression()
log3.fit(x_train,y_train)
log3.score(x_test,y_test)
print(classification_report (y_test, log3.predict(x_test)))

              precision    recall  f1-score   support

           0       0.98      0.83      0.90      1857
           1       0.52      0.90      0.66       377

    accuracy                           0.84      2234
   macro avg       0.75      0.87      0.78      2234
weighted avg       0.90      0.84      0.86      2234



In [56]:
log3.score(x_test,y_test)

0.8433303491495077

## Perfomance of Logistic regression model (4515-6020)

In [50]:
x_train, y_train = get_train_batch(df_class0, df_class1,4515, 6020)
from sklearn.linear_model import LogisticRegression
log4 = LogisticRegression()
log4.fit(x_train,y_train)
log4.score(x_test,y_test)
print(classification_report (y_test, log4.predict(x_test)))

              precision    recall  f1-score   support

           0       0.98      0.83      0.90      1857
           1       0.52      0.92      0.66       377

    accuracy                           0.84      2234
   macro avg       0.75      0.87      0.78      2234
weighted avg       0.90      0.84      0.86      2234



In [55]:
log4.score(x_test,y_test)

0.8424350940017905

## Perfomance of Logistic regression model (6020 - 7428)

In [53]:
x_train, y_train = get_train_batch(df_class0, df_class1,4515, 6020)
from sklearn.linear_model import LogisticRegression
log5 = LogisticRegression()
log5.fit(x_train,y_train)
log5.score(x_test,y_test)
print(classification_report (y_test, log5.predict(x_test)))

              precision    recall  f1-score   support

           0       0.98      0.83      0.90      1857
           1       0.52      0.92      0.66       377

    accuracy                           0.84      2234
   macro avg       0.75      0.87      0.78      2234
weighted avg       0.90      0.84      0.86      2234



In [54]:
log5.score(x_test,y_test)

0.8424350940017905